# Assignment 4: Variational Diffusion Models

*Author:* Eric Volkmann / Thomas Adler

*Copyright statement:* This  material,  no  matter  whether  in  printed  or  electronic  form,  may  be  used  for  personal  and non-commercial educational use only.  Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors. 

## Background:

In general latent variable models, we model the latent variables and the observations (data points) by a joint probability distribution $p(x, z)$. In the "likelihood-based" generative modelling setting, we aim to learn a model which maximizes the likelihood $p(x)$ of all data points. If the model is analytically tractable, we can explictly marginalize out the latent variables $z$:

$ p(x) = \int p(x, z) dz $

We derived the maximum likelihood equation for the pPCA model in Assignment 2. For more complex, intractable models this approach fails as we don't have access to the ground truth latent encoder $p(x, z)$.

In the last exercise sheet, we instead derived "Evidence lower bound" (ELBO), which is a lower bound on the evidence and that we can use as proxy objective to optimize our latent variable model, e.g. a simple VAE.

$\log p(x) \geq \mathbb{E} [ \log \frac{p(x,z)}{q_{\phi}(z|x)} ]
            = \underbrace{\mathbb{E}_{q_{\phi}(z|x)} [ \log p_{\theta} (x | z) ]}_{\text{reconstruction term}} - \underbrace{D_{KL} ( q_{\phi}(z | x) || p(z) )}_{\text{prior matching term}}$

In the variational autoencoder case, we learn an intermediate bottlenecking distribution $q_{\phi}(z|x)$ that can be treated as an encoder; it transforms inputs into a distribution over possible latents. Simultaneously, we learn a deterministic function $p_{\theta}(x|z)$ to convert a given latent vector $z$ into an observation $x$, which can be interpreted as a decoder.

In [3]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.imgur.com/rBXVRvI.png")

A Hierarchical Variational Autoencoder (HVAE) is a generalization of a VAE that extends to multiple hierarchies over latent variables. Whereas in the general HVAE with $T$ hierarchical levels, each latent is allowed to condition on all previous latents, we focus on a special case which we call a Markovian HVAE (MHVAE). In a MHVAE,
the generative process is a Markov chain; that is, each transition down the hierarchy is Markovian. The generative
process is modeled as a Markov chain, where each latent $z_t$ is generated only from the previous latent $z_{t+1}$.
Intuitively, and visually, this can be seen
as simply stacking VAEs on top of each other. Mathematically, we represent the joint distribution and the posterior of a
Markovian HVAE as:

$p(x, z_{1:T}) = p(z_{T}) p_{\theta}(x|z_1) \prod_{t=2}^{T} p_{\theta}(z_{t-1}|z_{t})$

$q_{\phi}(z_{1:T}|x) = q_{\theta} (z_1 | x) \prod_{t=2}^{T} q_{\theta} (z_{t}|z_{t-1})$

In [4]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.imgur.com/teD56gK.png")

The easiest way to think of a Variational Diffusion Model (VDM) is simply as a Markovian Hierarchical Variational Autoencoder with three key restrictions:

 i) The latent dimension is exactly equal to the data dimension
 
 ii) The structure of the latent encoder at each timestep is not learned; it is pre-defined as a linear Gaussian
model. In other words, it is a Gaussian distribution centered around the output of the previous timestep

 iii) The Gaussian parameters of the latent encoders vary over time in such a way that the distribution of
the latent at final timestep T is a standard Gaussian

From the second assumption, we know that the distribution of each latent variable in the encoder is a
Gaussian centered around its previous hierarchical latent.
The main takeaway is that $\alpha_t$ is a (potentially learnable) coefficient that can
vary with the hierarchical depth $t$, for flexibility. Mathematically, encoder transitions are denoted as:

\begin{equation*}
    q(x_t|x_{t-1}) = \mathcal{N}(x_t; \sqrt{\alpha_t} x_{t-1}, (1-\alpha_t) \textbf{I})
\end{equation*}

Note that our encoder distributions $q(x_t|x_{t−1})$ are no longer parameterized by $\phi$, as they are completely
modeled as Gaussians with defined mean and variance parameters at each timestep.

In [12]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.imgur.com/VOSpXkR.png")

## Exercise 1) The forward diffusion process (1 pts)

### Exercise 1a) Simulate the forward diffusion process (0.5 pts)

From the first restriction, with some abuse of notation, we can now represent both true data samples and latent variables as $x_t$, where $t = 0$ represents true data samples and $t \in [1, T ]$ represents a corresponding latent with hierarchy indexed by $t$. 

The "encoding process" is referred to as the noise corruption or forward diffusion process in diffusion models

and is defined as
\begin{align*}
    q(x_t \mid x_{t-1}) &= \mathcal N(\sqrt \alpha_t x_{t-1}, (1-\alpha_t) I ) && \text{where} & \alpha_t &\in (0, 1) \subset \mathbb R. 
\end{align*}
If $\alpha_t$ is close to 1, we draw $x_t$ from a normal distribution with mean close to $x_{t-1}$ and small variance. 
Simulate and visualize the forward diffusion process using a sample image from the CIFAR10 dataset for $T=500$. 


Use a fixed $\alpha$ for all time steps. By visual inspection, tune $\alpha$ to a value that lets $x_0$ smoothly blend into noise throughout the sequence.

In [1]:
#import math
from math import sqrt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as tfm
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):
    img = (img + 1) / 2 # rescale from [-1, 1] to [0, 1]
    plt.imshow(np.transpose(img.numpy().clip(0, 1), (1, 2, 0)))
    plt.show()

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=tfm.ToTensor())
x = trainset[0][0] * 2 - 1 # scale to [-1, 1]

Files already downloaded and verified


In [ ]:
########## YOUR SOLUTION HERE ##########

### Exercise 1b: Reparametrization trick for the forward process (0.5 pts)

Again, the noise corruption or forward diffusion process, using a fixed $\alpha$ across all timesteps, is defined as
\begin{align*}
    q(x_t \mid x_{t-1}) &= \mathcal N(x_t; \sqrt \alpha x_{t-1}, (1-\alpha) I ) && \text{where} & \alpha &\in (0, 1) \subset \mathbb R. 
\end{align*}
Use the reparameterization trick to show that
\begin{align*}
    q(x_t \mid x_0) = \mathcal N(x_t; \sqrt{\alpha^t} x_0, (1-\alpha^t) I).
\end{align*}
From this result, interpret the role of the parameter $\alpha$. 

########## YOUR SOLUTION HERE ##########

## Exercise 2: A ELBO for VDMs (1.5 pts)

In [8]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.imgur.com/juJ4KMv.png")

Note that our encoder distributions $q(x_{t}|x_{t−1})$ are no longer parameterized by $\varphi$, as they are completely
modeled as Gaussians with defined mean and variance parameters at each timestep. Therefore, in a VDM, we
are only interested in learning conditionals $p_{\theta} (x_{t-1} | x_{t})$, so that we can simulate new data. After optimizing
the VDM, the sampling procedure is as simple as sampling Gaussian noise from $p(x_T)$ and iteratively running
the denoising transitions $p_{\theta} (x_{t-1}|x_t)$ for $T$ steps to generate a novel $x_0$.

Like any HVAE, the VDM can be optimized by maximizing the ELBO. Derive the ELBO for the VDM and show that it written as follows:

\begin{equation*}
\log p(x_0) \geq \underbrace{\mathbb{E}_{q(x_1|x_0)}[\log p_{\theta}(x_0|x_1)]}_{\text{reconstruction term}}
- \underbrace{\mathbb{E}_{q(x_{T-1}|x_0)}[D_{KL}(q(x_{T}|x_{T-1} || p(x_{T}))]}_{\text{prior matching term}}
- \sum_{t=1}^{T-1} \underbrace{\mathbb{E}_{q(x_{t-1}, x_{t+1}|x_0)} [D_{KL} (q(x_t|x_{t-1}) || p_{\theta}(x_t|x_{t+1}))]}_{\text{consistency term}}
\end{equation*}
Interpret the differents terms.

########## YOUR SOLUTION HERE ##########

## Exercise 3: A smarter ELBO for VDMs (1.5 pts)

In [9]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.imgur.com/Q08pjU3.png")

The terms in the ELBO are expectations and are typcially computed using Monte Carlo estimates.

Unfortunately, optimizing the ELBO using the terms we just derived might be suboptimal: The consistency term is computed as an expectation over two random variables
${x_{t−1}, x_{t+1}}$ for every timestep and the variance of its Monte Carlo estimate could potentially be higher than a term that is estimated using only one random variable per timestep. As it is computed by summing up $T − 1$ consistency terms, the final estimated value of the ELBO may have high variance for large $T$ values.

The key insight is that we can rewrite encoder transitions as $q(x_{t}|x_{t−1}) = q(x_t|x_{t−1}, x_0)$, where the extra conditioning term is superfluous due to the Markov property. Then, according to Bayes rule, we can rewrite each transition as:

\begin{equation*}
    q(x_t|x_{t-1}, x_0)  = \frac{q(x_{t-1}|x_t, x_0) q(x_t|x_0)}{q(x_{t-1}|x_0)}
\end{equation*}

Retry the derivation of the ELBO using this equation and show that it can be written as follows:

\begin{equation*}
\log p(x_0) \geq \underbrace{\mathbb{E}_{q(x_1|x_0)}[\log p_{\theta}(x_0|x_1)]}_{\text{reconstruction term}}
- \underbrace{D_{KL}(q(x_{T}|x_{0}) || p(x_{T}))}_{\text{prior matching term}}
- \sum_{t=2}^{T} \underbrace{\mathbb{E}_{q(x_{t}|x_0)} [D_{KL} (q(x_{t-1}|x_{t},x_0) || p_{\theta}(x_{t-1}|x_{t}))]}_{\text{denoising matching term}}
\end{equation*}

Again, provide interpretations for the terms and argue if and why this is a better or worse optimization target than the previous ELBO.

########## YOUR SOLUTION HERE ##########

## Exercise 4: Backward Diffusion Process (2 pts)

To instantiate this objective we need to determine the distribution $q(x_{t-1} \mid x_t, x_0)$, which can be thought of as the backward diffusion process.

We know that $q(x_t|x_{t-1}) = \mathcal{N}(x_t; \sqrt{\alpha_t} x_{t-1}, (1-\alpha_t) \textbf{I})$


Show that 
\begin{align*}
    q(x_{t-1} \mid x_t, x_0) = \mathcal N\left(x_{t-1}; \frac1{1-\alpha^t} \left( \sqrt \alpha (1-\alpha^{t-1}) x_t + \sqrt{\alpha^{t-1}} (1-\alpha) x_0 \right), \frac{(1-\alpha)(1-\alpha^{t-1})}{1-\alpha^t}\right).
\end{align*}

Assume constant $\alpha = \alpha_t$  $\forall t \in [0, T]$.

*Hint: Use the result for Exercise 1b) and the conditional version of Bayes' theorem*

########## YOUR SOLUTION HERE ##########

## Exercise 5: Training (2 pts)

Now that we know $q(x_{t-1} \mid x_t, x_0)$, we can construct $p_\theta(x_{t-1} \mid x_t)$ by letting our model predict $x_0$ and plugging that prediction into the solution for $q(x_{t-1} \mid x_t, x_0)$. 

Since both $q(x_{t-1} \mid x_t, x_0)$ and $p_\theta(x_{t-1} \mid x_t)$ are Gaussian, maximizing the ELBO results in minimizing the mean-squared error. 

Implement a training loop for the network specified below. 

Do a uniform Monte-Carlo estimate over $t$, i.e., for each sample, draw a time step $t$ uniformly from ${1, \dots, T}$ and corrupt the image accordingly to obtain $x_t$. 
Then train the VDM to predict $\varepsilon$ from $x_t$, i.e. minimize the MSE between the ground truth image $x_0$ and the predicted image $\hat{x}_0$.

For efficiency reasons, we share the model parameters over time. 
To improve performance, we feed $t$ as an additional input alongside $x_t$. 
The provided hyperparameters should work to get "ok" results but feel free to experiment with them as always. 

In [ ]:
batch_size = 128
epochs = 200
lr = 1e-3
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=tfm.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=tfm.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

class VDM(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(4, 32, 4, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, 4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.deconv1 = nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1)
        self.debn1 = nn.BatchNorm2d(128)
        self.deconv2 = nn.ConvTranspose2d(256, 64, 4, stride=2, padding=1)
        self.debn2 = nn.BatchNorm2d(64)
        self.deconv3 = nn.ConvTranspose2d(128, 32, 4, stride=2, padding=1)
        self.debn3 = nn.BatchNorm2d(32)
        self.deconv4 = nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1)
        self.debn4 = nn.BatchNorm2d(3)

    def forward(self, x, t):
        t = torch.ones_like(x[:, :1, :, :]) * (t/T * 2 - 1)
        x0 = torch.cat([x, t], 1)
        x1 = F.relu(self.bn1(self.conv1(x0)))
        x2 = F.relu(self.bn2(self.conv2(x1)))
        x3 = F.relu(self.bn3(self.conv3(x2)))
        x4 = F.relu(self.bn4(self.conv4(x3)))
        x4 = F.relu(self.debn1(self.deconv1(x4)))
        x3 = F.relu(self.debn2(self.deconv2(torch.cat([x4, x3], 1))))
        x2 = F.relu(self.debn3(self.deconv3(torch.cat([x3, x2], 1))))
        return F.tanh(self.debn4(self.deconv4(torch.cat([x2, x1], 1))))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
vdm = VDM().to(device)
vdm.train()
optimizer = torch.optim.Adam(vdm.parameters(), lr=lr)
print(torch.nn.utils.parameters_to_vector(vdm.parameters()).numel(), 'parameters')

########## YOUR SOLUTION HERE ##########

## Bonus Exercise: Inference and Visualization (1 pts)

The key idea of diffusion models is to generate images by performing the backward diffusion process using our trained model to predict $x_0$. 
That is, at time $t$, sample $x_{t-1}$ from  
\begin{align*}
    p(x_{t-1} \mid x_t, \hat x_0) = \mathcal N\left(\frac1{1-\alpha^t} \left( \sqrt \alpha (1-\alpha^{t-1}) x_t + \sqrt{\alpha^{t-1}} (1-\alpha) \hat x_0 \right), \frac{(1-\alpha)(1-\alpha^{t-1})}{1-\alpha^t} I\right)
\end{align*}
where $\hat x_0$ is the model prediction from $x_t$. 

Iterate this scheme from $t=T$ until $t=0$ to generate a bunch of sample images.

Visualize the denoising process of going from pure noise to a sample image (inverse of exercise 1a). Plot either a sequence of images or make a short animation.

In [ ]:
########## YOUR SOLUTION HERE ##########